# Permanent randomized response

The basic idea of differential privacy is to make a particular query stochastic so that the underlying data is kept private. The average attack consists of performing the same query many times in order to reliably estimate the underlying data. This is, of course, not desirable so we should either limit the number of queries or design algorithms that are not vulnerable under this kind of attack.

In this notebook we present a simple example of this phenomenon based on a single node that contains one binary number $n$ that encodes whether the node is guilty ($n=1$) or innocent ($n=0$). 
A randomized response algorithm (see [The Algorithmic Foundations of Differential Privacy](https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf), Section 3.2, and the notebook [Basic concepts](./differential_privacy_basic_concepts.ipynb)) is used to query the node in order to preserve the privacy of $n$.

### Average attack

We start by creating a single node that contains a binary number. In this case, we set this number to 1 (guilty). By setting f1=0.8, we make sure that 80% of the times we query the node whose data is 1, we get an answer of 1. On the remaining 20% of the cases we obtain an answer of 0.

In [ ]:
import numpy as np
from shfl.private.node import DataNode
from shfl.private.node import ExceededPrivacyBudgetError
from shfl.differential_privacy.dp_mechanism import RandomizedResponseBinary
from math import log, exp

n = 1 #the node is guilty

global_epsilon_delta = epsilon_delta=(70, 0)

node_single = DataNode(global_epsilon_delta)
node_single.set_private_data(name="guilty", data=np.array([n]))
data_access_definition = RandomizedResponseBinary(f0=0.8, f1=0.8, epsilon=log(4) + 1e-10)
node_single.configure_data_access("guilty", data_access_definition)

If we perform the query just once, we cannot be sure that the result matches the true data.

In [ ]:
result = node_single.query(private_property="guilty")
print("The result of one query is: " + str(int(result)))

If we perform the query $N$ times and take the average, we can estimate the true data with an error that goes to zero as $N$ increases.

In [ ]:
N = 50
result_query = []
for i in range(N):
    try:
        result_query.append(node_single.query(private_property="guilty"))
    except ExceededPrivacyBudgetError:
        print("The privacy budget has been spent at run {}".format(i+1))
        break
result_query = np.array(result_query)
print(np.mean(result_query))

We see that the average result of the query is close to 0.8. This allows us to conclude that the raw answer is most probably 1. Otherwise, the result would've been close to 0.2.

### Permanent randomized response

A possible solution to this problem (e.g. see [RAPPOR technology](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/42852.pdf) and [Jiang 2019](https://www.sciencedirect.com/science/article/pii/S0020025518310429)) is to create a node that contains two pieces of information: the true data and a **permanent randomized response** (PRR). The latter is initialized to None and, once the node receives the first query, it creates a random binary number following the algorithm described above which is saved as the PRR. The result of the query is then a randomized response using the PRR as input. This way, even if the query is performed a large number of times, the attacker may only guess with certainty the PRR, but not the true data.

In [ ]:
node_single_prr = DataNode(global_epsilon_delta)
data_binary = np.array([1])  #the node is guilty
node_single_prr.set_private_data(name="guilty", data=np.array([n]))
node_single_prr.configure_data_access("guilty", data_access_definition)

permanent_response = node_single_prr.query(private_property="guilty")
print("The PRR is: " + str(int(permanent_response)))

# we save the prr as a new piece of information
node_single_prr.set_private_data(name="guilty", data=np.append(data_binary, permanent_response))

From now on, all the external queries are done over the permanent randomized data, while the raw data remains completely hidden.

In [ ]:
N = 50
result_query = []
for i in range(N):
    try:
        result_query.append(node_single_prr.query(private_property="guilty")[1])
    except ExceededPrivacyBudgetError:
        print("The privacy budget has been spent at run {}".format(i+1))
        break
result_query = np.array(result_query)
print(np.mean(result_query))

The result is not always close to 0.8, since the permanent response might be 0. The average attack may, at best, identify the permanent randomized response, but not the raw data.